In [ ]:
%pip install -r requirements.txt

In [ ]:
from dotenv import load_dotenv  
import os  

from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage
  
# Load the .env file  
load_dotenv()  
  
# Get an environment variable  
endpoint = os.getenv('AZURE_OPENAI_ENDPOINT') 
key = os.getenv('AZURE_OPENAI_API_KEY')
api_version = os.getenv('AZURE_OPENAI_API_VERSION')  
chat_deployment = os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT')  


chat = AzureChatOpenAI(
    azure_deployment=chat_deployment,
    openai_api_version=api_version,
)

# Test chat openai
message = HumanMessage(
    content="Translate this sentence from English to French. I love programming."
)
result = chat.invoke([message]) 
print(result.content)  
  

In [ ]:
import bs4
from langchain import hub

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_openai import AzureOpenAIEmbeddings
  

# specify the directory where the index will be saved/loaded from  
directory = "./chroma_db"



# Get an environment variable  
embedding_deployment = os.getenv('AZURE_OPENAI_EMBEDDING_DEPLOYMENT')


embedding=AzureOpenAIEmbeddings(
    azure_deployment=embedding_deployment,
    openai_api_version=api_version,)



  
if os.path.exists(directory):  
    print(f"Index exists, loading") 
    # if the directory exists, load the index  
    vectorstore = Chroma(persist_directory=directory, embedding_function=embedding)

else:  
    print(f"Index not exist, building") 
    # if the directory does not exist, create the index and save it  

    # Load, chunk and index the contents of the blog.
    loader = WebBaseLoader(
        web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
        bs_kwargs=dict(
            parse_only=bs4.SoupStrainer(
                class_=("post-content", "post-title", "post-header")
            )
        ),
    )
    docs = loader.load()


    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(docs)

    vectorstore = Chroma.from_documents(documents=splits, embedding=embedding, persist_directory=directory)  


In [ ]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | chat
    | StrOutputParser()
)
print(rag_chain.invoke("What is Task Decomposition?"))